## Импорт библиотек 

In [12]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.optimize as opt
import ot  # Для транспортной метрики
from scipy.stats import uniform, truncnorm
import time
from itertools import permutations

C:\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
C:\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [13]:
%matplotlib qt

## Описание функций

In [1]:
def plot_3d(C):
    # Create figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    ax.set_zlim(22, 0)

    # Plot the voxels where value == 1
    ax.voxels(C, edgecolor='k')

In [2]:
def compare_XY(X, Y):
    # Create figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    ax.set_zlim(22, 0)

    # Plot the voxels where value == 1
    ax.voxels(X, edgecolor='k')
    ax.voxels(Y, edgecolor='r')

In [3]:
def compare_proj(X, Y):
    # Create figure and 3D axis
    fig, ax = plt.subplots(1, 2)
    
    ax[0].matshow(X.any(axis=0).transpose() + 5 * Y.any(axis=0).transpose() , cmap='Greys')
    ax[1].matshow(X.any(axis=1).transpose() + 5 * Y.any(axis=1).transpose() , cmap='Greys')
    
    ax[0].set_aspect(96 / 22)
    ax[0].set_xlim(0, 96)
    ax[0].set_ylim(22, 0)
    ax[1].set_aspect(96 / 22)
    ax[1].set_xlim(0, 96)
    ax[1].set_ylim(22, 0)

In [4]:
def x_proj(C):
    return C.any(axis=1)

def y_proj(C):
    return C.any(axis=0)

In [5]:
def plot_X_projection(C):
    plt.matshow(C.any(axis=1).transpose(), cmap='Greys')

def plot_Y_projection(C):
    plt.matshow(C.any(axis=0).transpose(), cmap='Greys')
    
def plot_projections(C):
    fig, ax = plt.subplots(1, 2)
    
    ax[0].matshow(C.any(axis=1).transpose(), cmap='Greys')
    ax[1].matshow(C.any(axis=0).transpose(), cmap='Greys')
    
    ax[0].set_aspect(96 / 22)
    ax[0].set_xlim(0, 96)
    ax[0].set_ylim(22, 0)
    ax[1].set_aspect(96 / 22)
    ax[1].set_xlim(0, 96)
    ax[1].set_ylim(22, 0)

In [6]:
def check_XY_bounds(x, xmin=0, xmax=95):
    return (x >= xmin) & (x <= xmax)

## Процедуры генерации событий

Процедура генерации события состоит в следующем.

Есть точка влёта **start_x, start_y**; углы влёта **start_theta, start_phi**; плоскость взаимодействия **zint** (индексация с нуля), количество порождённых частиц, и два массива с углами разлёта частиц **theta_part, phi_part**.

**start_theta** принимает значения от 0 до pi/3 (иначе просто не попадёт в апертуру прибора);

**theta_part** --- от 0 до pi, т.к. порождённая частица может "развернуться" (полететь в обратную сторону).

**phi** везде от -pi до pi.

In [41]:
# StartX 0..95, StartY 0..95, start_theta 0..?, start_phi -pi..pi
def _generate_event(startx, starty, start_theta, start_phi, zint, npart, theta_part, phi_part):
    startz = 0
    maxz = 21
    zint = int(zint)
    l = (np.tan(start_theta) * np.cos(start_phi), np.tan(start_theta) * np.sin(start_phi), 1)  # Направляющий вектор прямой с координатой z = 1
    lx = startx + l[0] * np.arange(0, zint + 1, 1)
    ly = starty + l[1] * np.arange(0, zint + 1, 1)
    lz = np.arange(0, zint + 1, 1, dtype=int)
    xint, yint = lx[-1], ly[-1]  # Координаты X, Y точки взаимодействия
    
    C = np.zeros((96, 96, 22), dtype=int)
    
    # Флаг прерванного трека (вылет за пределы калориметра) 
    track_interrupted = False
    
    # Проверка того, вышли ли мы за пределы калориметра
    # Если вылетели, то обрываем траекторию
    if not(check_XY_bounds(xint) and check_XY_bounds(yint)): 
        idx = check_XY_bounds(lx) & check_XY_bounds(ly)
        lx, ly, lz = lx[idx], ly[idx], lz[idx]
        track_interrupted = True
    
    lx_int = np.round(lx).astype(int)
    ly_int = np.round(ly).astype(int)
    
    C[lx_int, ly_int, lz] = 1  # Добавили траекторию первичной частицы
    
    
    if not(track_interrupted):
        lines = dict()
        
        direction = np.array(theta_part) < np.pi / 2

        for line_num in range(npart):
            newline = []
            if direction[line_num]:
                steps = maxz - zint + 1
                newline = [
                    xint + np.tan(theta_part[line_num]) * np.cos(phi_part[line_num]) * np.arange(0, steps - 1, 1),
                    yint + np.tan(theta_part[line_num]) * np.sin(phi_part[line_num]) * np.arange(0, steps - 1, 1),
                    np.arange(zint, maxz, 1, dtype=int)
                ]
            else:
                steps = zint + 1
                newline = [
                    xint - np.tan(theta_part[line_num]) * np.cos(phi_part[line_num]) * np.arange(0, steps, 1),
                    yint - np.tan(theta_part[line_num]) * np.sin(phi_part[line_num]) * np.arange(0, steps, 1),
                    np.arange(zint, -1, -1, dtype=int)
                ]
            # Отрезаем значения вне прибора
            idx = (newline[0] >= 0) & (newline[0] <= 95) & (newline[1] >= 0) & (newline[1] <= 95)
            lines[line_num] = [newline[0][idx], newline[1][idx], newline[2][idx]]



        for line_num in range(npart):
            C[np.round(lines[line_num][0]).astype(int),
              np.round(lines[line_num][1]).astype(int),
              lines[line_num][2]] = 1
    
    return C

In [8]:
# Создаём событие для N порождённых частиц по списку параметров
# Универсальная
def _generate_N_event(params, N):
    return _generate_event(*params[0:5], N, params[5: 5 + N], params[5 + N: 5 + 2*N])

In [55]:
# Событий с точкой взаимодействия в середине должно быть больше
def generate_random_startx(size=100):
    loc, scale = 47.5, 20.0
    # Define bounds (in standard normal space)
    lower, upper = -loc / scale, loc / scale
    samples = truncnorm.rvs(lower, upper, loc=loc, scale=scale, size=size)
    integers = np.round(samples)
    return integers

# Событий с точкой взаимодействия в середине должно быть больше
def generate_random_zint(size=100):
    # Define bounds (in standard normal space)
    lower, upper = (0 - 10.5) / 4.0, (21 - 10.5) / 4.0
    samples = truncnorm.rvs(lower, upper, loc=10.5, scale=4.0, size=size)
    integers = np.round(samples).astype(int)
    return integers

# Азимутальный угол распределён равномерно
def generate_random_phi_angle(size=100):
    samples = uniform.rvs(-np.pi, np.pi, size=size)
    return samples

# Зенитный угол распределён равномерно
def generate_random_theta_start_angle(size=100):
    scale = 0.3
    # Define bounds (in standard normal space)
    lower, upper = -np.pi / 3 / scale, np.pi / 3 / scale
    samples = truncnorm.rvs(lower, upper, loc=0, scale=scale, size=size)
    return np.abs(samples)

# Зенитный угол распределён равномерно
def generate_random_theta_int_angle(size=100):
    samples = uniform.rvs(0, np.pi, size=size)
    return samples

In [31]:
# Выборка случайных событий с фиксированным количеством порожденных частиц
def generate_random_events(n_part, size=100):
    startx = generate_random_startx(size=size)
    starty = generate_random_startx(size=size)
    theta = generate_random_theta_start_angle(size=size)
    phi = generate_random_phi_angle(size=size)
    zint = generate_random_zint(size=size)
    theta_part = generate_random_theta_int_angle(size * n_part)
    phi_part = generate_random_phi_angle(size * n_part)
    
    return [_generate_event(startx[i], starty[i], theta[i], phi[i], zint[i], n_part, theta_part[i * n_part: (i + 1) * n_part], phi_part[i * n_part: (i + 1) * n_part]) for i in range(size)]
    

## Определим метрику расстояния и целевую функцию

In [48]:
# Будем минимизировать транспортную метрику (Вассерштайна)
# Она энергозатратная, но для разреженных матриц вроде бы не критично
def wasserstein_distance(mat1, mat2):
    # Get coordinates of 1s in each matrix
    coords1 = np.argwhere(mat1 == 1)  # shape (N, 3)
    coords2 = np.argwhere(mat2 == 1)  # shape (M, 3)
    
    if len(coords1) == 0 or len(coords2) == 0:
        distance = np.inf
    else:
        # Compute Euclidean cost matrix (distance between all pairs)
        cost_matrix = ot.dist(coords1, coords2, metric='euclidean')

        # Uniform weights (since all 1s are equally important)
        weights1 = np.ones(len(coords1)) / len(coords1)
        weights2 = np.ones(len(coords2)) / len(coords2)

        # Compute Wasserstein distance
        distance = ot.emd2(weights1, weights2, cost_matrix)
    return distance

In [47]:
# Расстояние Хэмминга (симметрическая разность)
def hamming_distance(mat1, mat2):
    return np.sum(mat1 != mat2)

In [46]:
# Функция для сравнения со случаем, когда порождены N частиц (N >= 1) --- универсальная
def _objective_N(params, to_x, to_y, N):
    startx, starty, theta, phi, zint, N, theta_part, phi_part = *params[0:5], N, params[5: 5 + N], params[5 + N: 5 + 2*N]
    E = _generate_event(startx, starty, theta, phi, zint, N, theta_part, phi_part)
    return wasserstein_distance(to_x, x_proj(E)) + wasserstein_distance(to_y, y_proj(E))

## Восстанавливаем трек частицы

In [176]:
# %store -r good_test_event

Stored 'good_test_event' (ndarray)


In [164]:
test_events = generate_random_events(7, 100)

In [165]:
for i in range(20, 30):
    plot_3d(test_events[i])

In [169]:
plot_3d(test_events[23])

In [175]:
# good_test_event = test_events[23]

In [170]:
event = test_events[23]
X, Y = x_proj(event), y_proj(event)

# В этом алгоритме есть стартовая точка, её надо выбрать
start_x, start_y = np.argwhere(X)[0][0], np.argwhere(Y)[0][0]

# Сначала быстрым методом выберем стартовую точку (здесь надо ветки как-то поравномернее взять)
particle_num = 7
start_theta_part = generate_random_theta_int_angle(size=particle_num)
start_phi_part = generate_random_phi_angle(size=particle_num)

start_result = opt.minimize(_objective_N, x0=[start_x, start_y, 0.0, 0.0, 10.0, *start_theta_part, *start_phi_part], args=(X, Y, particle_num),
                            bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), *[(0, np.pi)] * particle_num, *[(-np.pi, np.pi)] * particle_num],
                            callback=lambda result: print(".", end=""),
                            method='Nelder-Mead')

print("Differential evolution...")

def diff_callback(xk, convergence):
    current_min = _objective_N(xk, X, Y, particle_num)
    print(r"{0:.3f} / ".format(current_min), end="")
    return False
    return current_min < 0.001  # Stop if condition met

result = opt.differential_evolution(_objective_N, args=(X, Y, particle_num),
                      x0 = start_result.x,
                      init = 'sobol',
                      bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), *[(0, np.pi)] * particle_num, *[(-np.pi, np.pi)] * particle_num],
                      callback=diff_callback,
                      maxiter=2000,
                      tol=1e-3)

.............................................................................................................................................................................................................................................................Differential evolution...
8.117 / 8.117 / 6.700 / 5.537 / 5.537 / 4.774 / 4.774 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.025 / 4.004 / 4.004 / 4.004 / 4.004 / 4.004 / 3.332 / 3.332 / 3.332 / 3.332 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 3.316 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.637 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.513 / 2.420 / 2.420 / 2.420 / 2.420 / 2.420 / 2

Заметим, что возможны симметрии, не меняющие проекции. Например, если три частицы летят вниз, у нас 3! = 6 вариантов исходного события.<br>Смысл такой: допустим, у нас есть две прямые с направляющими векторами (x1, y1, 1) и (x2, y2, 1), то пара прямых с напр. векторами (x1, y2, 1) и (x2, y1, 1) будет проецироваться так же. Это надо учесть, когда будем проводить оценку работы алгоритма на известных данных.

In [171]:
# Восстановленное событие
hamming_distance(event, _generate_N_event(result.x, N=particle_num))

45

In [149]:
def cartesian_to_spherical(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
#     theta = np.arccos(z / r) if r != 0 else 0.0
    theta = np.arccos(z / r)
    phi = np.arctan2(y, x)
    return r, theta, phi

In [112]:
def spherical_to_cartesian(r, theta, phi):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z

In [172]:
# Теперь переставим углы phi (theta оставим на месте).
theta_angles = result.x[5: 5 + particle_num]
phi_angles = result.x[5 + particle_num: 5 + 2 * particle_num]

In [173]:
theta_angles

array([0.80077293, 0.67454539, 2.19936613, 1.99600843, 0.85560472,
       3.10092773, 0.01325009])

In [174]:
phi_angles

array([-2.79256337, -0.80187626, -0.96257514, -1.92325814, -0.17834008,
        0.46303997, -0.69880314])

In [183]:
# Направляющие векторы получившихся прямых.
# Нормируем на координату Z. Если знак Z совпадает, то мы можем перепутать прямые при проецировании.
x_dir, y_dir, z_dir = spherical_to_cartesian(np.ones(particle_num), theta_angles, phi_angles)
x_dir, y_dir, z_dir = x_dir / np.abs(z_dir), y_dir / np.abs(z_dir), z_dir / np.abs(z_dir)

print(x_dir, y_dir, z_dir)

[-0.96905418  0.55606466  0.78606097 -0.7623229   1.13302288  0.03640292
  0.01014503] [-0.35266675 -0.57469918 -1.12895284 -2.07253857 -0.20423323  0.01817382
 -0.0085243 ] [ 1.  1. -1. -1.  1. -1.  1.]


Теперь мы можем "перепутать" векторы, для которых z_dir совпадает.

In [192]:
# Возьмём индексы прямых, направленных вперёд (Z = 1)
fwd_idx = z_dir > 0
fwd_list = np.where(fwd_idx)[0]
fwd_permutations = list(permutations(fwd_list))
print(fwd_permutations)

# И индексы прямых, направленных назад (Z = -1)
bwd_idx = z_dir < 0
bwd_list = np.where(bwd_idx)[0]
bwd_permutations = list(permutations(bwd_list))
print(bwd_permutations)

[(0, 1, 4, 6), (0, 1, 6, 4), (0, 4, 1, 6), (0, 4, 6, 1), (0, 6, 1, 4), (0, 6, 4, 1), (1, 0, 4, 6), (1, 0, 6, 4), (1, 4, 0, 6), (1, 4, 6, 0), (1, 6, 0, 4), (1, 6, 4, 0), (4, 0, 1, 6), (4, 0, 6, 1), (4, 1, 0, 6), (4, 1, 6, 0), (4, 6, 0, 1), (4, 6, 1, 0), (6, 0, 1, 4), (6, 0, 4, 1), (6, 1, 0, 4), (6, 1, 4, 0), (6, 4, 0, 1), (6, 4, 1, 0)]
[(2, 3, 5), (2, 5, 3), (3, 2, 5), (3, 5, 2), (5, 2, 3), (5, 3, 2)]


In [213]:
result_permutations_events = []

counter = 0
# Поменяем Y у направляющих векторов, переведём в сферические и создадим новые события.
for i in range(len(fwd_permutations)):
    for j in range(len(bwd_permutations)):
        y_dir_new = np.zeros(particle_num)
        
        for k in range(len(fwd_permutations[i])):
            y_dir_new[fwd_list[k]] = y_dir[fwd_permutations[i][k]]
            
        for k in range(len(bwd_permutations[j])):
            y_dir_new[bwd_list[k]] = y_dir[bwd_permutations[j][k]]
            
        r, theta_angles_new, phi_angles_new = cartesian_to_spherical(x_dir, y_dir_new, z_dir)
        result_permutations_events += [_generate_N_event(np.concatenate([result.x[:5], theta_angles_new, phi_angles_new]), N=particle_num)]
        print(counter, *result.x[:5], theta_angles_new, phi_angles_new)
        counter += 1

0 54.226915347224946 75.34471799138745 0.0529634191885483 -0.6693961835145646 11.883108594626533 [0.80077293 0.67454539 2.19936613 1.99600843 0.85560472 3.10092773
 0.01325009] [-2.79256337 -0.80187626 -0.96257514 -1.92325814 -0.17834008  0.46303997
 -0.69880314]
1 54.226915347224946 75.34471799138745 0.0529634191885483 -0.6693961835145646 11.883108594626533 [0.80077293 0.67454539 2.19936613 2.49011445 0.85560472 2.02028587
 0.01325009] [-2.79256337 -0.80187626 -0.96257514  3.11775711 -0.17834008 -1.55323372
 -0.69880314]
2 54.226915347224946 75.34471799138745 0.0529634191885483 -0.6693961835145646 11.883108594626533 [0.80077293 0.67454539 1.99459932 2.20403729 0.85560472 3.10092773
 0.01325009] [-2.79256337 -0.80187626 -1.20828345 -2.1647162  -0.17834008  0.46303997
 -0.69880314]
3 54.226915347224946 75.34471799138745 0.0529634191885483 -0.6693961835145646 11.883108594626533 [0.80077293 0.67454539 1.99459932 2.49011445 0.85560472 2.29543949
 0.01325009] [-2.79256337 -0.80187626 -1.208

In [211]:
for i in range(len(result_permutations_events)):
    print(i, hamming_distance(event, result_permutations_events[i]))

0 45
1 85
2 89
3 105
4 106
5 82
6 59
7 99
8 103
9 119
10 120
11 96
12 24
13 64
14 68
15 84
16 85
17 61
18 59
19 99
20 103
21 119
22 120
23 96
24 42
25 82
26 86
27 102
28 103
29 79
30 63
31 103
32 107
33 123
34 124
35 100
36 36
37 76
38 80
39 96
40 97
41 73
42 50
43 90
44 94
45 110
46 111
47 87
48 46
49 86
50 90
51 106
52 107
53 83
54 64
55 104
56 108
57 124
58 125
59 101
60 64
61 104
62 108
63 124
64 125
65 101
66 68
67 108
68 112
69 128
70 129
71 105
72 0
73 40
74 44
75 60
76 61
77 37
78 35
79 75
80 79
81 95
82 96
83 72
84 31
85 71
86 75
87 91
88 92
89 68
90 49
91 89
92 93
93 109
94 110
95 86
96 49
97 89
98 93
99 109
100 110
101 86
102 32
103 72
104 76
105 92
106 93
107 69
108 28
109 68
110 72
111 88
112 89
113 65
114 49
115 89
116 93
117 109
118 110
119 86
120 59
121 99
122 103
123 119
124 120
125 96
126 63
127 103
128 107
129 123
130 124
131 100
132 59
133 99
134 103
135 119
136 120
137 96
138 42
139 82
140 86
141 102
142 103
143 79


In [162]:
for i in range(len(result_permutations_events)):
    compare_XY(event, result_permutations_events[i])
    plt.savefig('{0}.png'.format(i))
    plt.close()

In [ ]:
good_test_event.tofile('test_event.txt', sep=" ", format="%d")

In [236]:
test_event = np.fromfile('test_event.txt', sep=" ", dtype=int).reshape(96, 96, 22)